<a href="https://colab.research.google.com/github/ambikad04/FL-PrivacyPreserving-HealthcareAnalytics/blob/main/PrivacyPreserving_HealthcareAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [23]:
# Define a Fully Connected Model for Tabular Data
class FCModel(nn.Module):
    def __init__(self, input_dim):
        super(FCModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Output layer for binary classification
        return x

In [24]:
# Create Synthetic Data (Healthcare)
X, y = make_classification(n_samples=100, n_features=10, n_informative=8, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
# Simulate Client Data (Split data into 5 clients)
clients_data = [(torch.tensor(X_train[i:i+20], dtype=torch.float32), torch.tensor(y_train[i:i+20], dtype=torch.float32)) for i in range(0, 80, 20)]
clients_data_test = [(torch.tensor(X_test[i:i+5], dtype=torch.float32), torch.tensor(y_test[i:i+5], dtype=torch.float32)) for i in range(0, 20, 5)]

In [26]:
# Train Local Model on Each Client
def train_local_model(model, data, targets, epochs=5):
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    return model.state_dict()

In [27]:
# Federated Averaging (with Personalization)
def federated_averaging_with_personalization(global_model, local_models_weights, personalization_factor=0.1):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict:
        global_state_dict[key] = torch.mean(torch.stack([local_weights[key] for local_weights in local_models_weights]), dim=0)
        for local_weights in local_models_weights:
            global_state_dict[key] = global_state_dict[key] * (1 - personalization_factor) + local_weights[key] * personalization_factor
    global_model.load_state_dict(global_state_dict)

In [28]:
# Initialize the Global Model
input_dim = X_train.shape[1]  # Number of features in the dataset
global_model = FCModel(input_dim)


In [29]:
# Federated Learning Process (10 rounds)
for round in range(10):
    print(f"Round {round + 1}")
    local_models_weights = []

    # Each client trains on its local data
    for client_data, client_targets in clients_data:
        local_model = FCModel(input_dim)
        local_model.load_state_dict(global_model.state_dict())  # Initialize with global model weights
        local_weights = train_local_model(local_model, client_data, client_targets)
        local_models_weights.append(local_weights)

    # Federated Averaging with Personalization
    federated_averaging_with_personalization(global_model, local_models_weights)

    # Testing the model on client data (optional)
    global_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for client_data, client_targets in clients_data_test:
            outputs = global_model(client_data)
            predicted = (outputs.squeeze() > 0.5).float()
            total += client_targets.size(0)
            correct += (predicted == client_targets).sum().item()
        accuracy = correct / total
        print(f"Test Accuracy: {accuracy * 100:.2f}%")

print("Training complete.")

Round 1
Test Accuracy: 45.00%
Round 2
Test Accuracy: 45.00%
Round 3
Test Accuracy: 45.00%
Round 4
Test Accuracy: 50.00%
Round 5
Test Accuracy: 50.00%
Round 6
Test Accuracy: 55.00%
Round 7
Test Accuracy: 60.00%
Round 8
Test Accuracy: 65.00%
Round 9
Test Accuracy: 70.00%
Round 10
Test Accuracy: 70.00%
Training complete.
